In [1]:
mlist = [
"bert-base-uncased",
"Yueh-Huan/news-category-classification-distilbert",
"abhishek/autonlp-bbc-news-classification-37229289"
]

import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import mlflow


df = pd.read_csv('complete_data_subset.csv')
df.kategorie_id = df.kategorie_id - 1
df = df.rename(columns={"kategorie_id":"label", "inhalt":"text"})
df = df.drop(columns=['kategorie_bezeichnung', 'titel', 'inhalt_kurz'])
df = df[df['text'].notna()]
traindf, testdf = train_test_split(df, test_size=0.2)

train = Dataset.from_pandas(traindf)
test = Dataset.from_pandas(testdf)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

for x in mlist:
  # %%
  print(x)
  # %%
  from transformers import AutoTokenizer
  model_name = x

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  # %%

  def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)

  tokenized_datasets = dataset.map(preprocess_function, batched=True)
  # %%
  from transformers import DataCollatorWithPadding

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  # %%
  from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

  model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                            num_labels=7,
                                                            ignore_mismatched_sizes=True)
  # %%
  import numpy as np
  from datasets import load_metric
  metric = load_metric("accuracy")
  #%%
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
  # %%
  num_epochs = 2

  args = TrainingArguments(
      output_dir ='./results',          
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 16,   
      per_device_eval_batch_size = 16,   
      weight_decay = 0.01,               
      learning_rate = 2e-5,             
      metric_for_best_model = 'accuracy',    
      evaluation_strategy = "epoch",
      )


  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets['train'],
      eval_dataset=tokenized_datasets['test'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )  
  mlflow.autolog()
  trainer.train()
  mlflow.log_param("model",x)

/opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 2/3150 [1:50:39<3397:03:47, 3884.82s/it]